<a href="https://colab.research.google.com/github/anoopjakob/AI_Lab_Internal_Exam/blob/main/AI_Lab_Internal_Exam.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

## Question 1
Implement the naive Bayes classifier for a sample training data set exam.CSV file without using any library that implements naive Bayes. Compute the accuracy of the classifier, considering few test data sets.

Submitted by **Anoop Jacob**

In [165]:
# All imports here

import numpy as np
import pandas as pd
import matplotlib.pyplot as plt

from math import sqrt
from math import exp
from math import pi


%matplotlib inline

In [166]:
# Loading data using pandas

columns = ['feature1', 'feature2', 'feature3', 'feature4', 'feature5', 
           'feature6', 'feature7', 'feature8','label']

exam_df = pd.read_csv('/content/drive/MyDrive/AI-ML/Lab_internal_exam/Anoop Jacob - exam.csv', usecols = columns)

# checking the shape
print(exam_df.shape)

# checking for null values
print(exam_df.info())

(768, 9)
<class 'pandas.core.frame.DataFrame'>
RangeIndex: 768 entries, 0 to 767
Data columns (total 9 columns):
 #   Column    Non-Null Count  Dtype  
---  ------    --------------  -----  
 0   feature1  768 non-null    int64  
 1   feature2  768 non-null    int64  
 2   feature3  768 non-null    int64  
 3   feature4  768 non-null    int64  
 4   feature5  768 non-null    int64  
 5   feature6  768 non-null    float64
 6   feature7  768 non-null    float64
 7   feature8  768 non-null    int64  
 8   label     768 non-null    int64  
dtypes: float64(2), int64(7)
memory usage: 54.1 KB
None


In [167]:
# SPLITTING DATA FOR TRAIN AND TEST

split_at_70 = int(len(exam_df)*0.7)

train_df = exam_df[:split_at_70]
train_dataset = train_df.values.tolist()
y_train = train_df.label
X_train = train_df.drop('label', axis=1).values.tolist()

test_df = exam_df[split_at_70:]
y_test = test_df.label
X_test = test_df.drop('label', axis=1).values.tolist()

In [168]:
len(y_test)
len(X_test)

231

In [169]:
# NAIVE BAYES FUNCTIONS CREATED HERE


# Split the dataset by class values, returns a dictionary
def separate_by_class(dataset):
	separated = dict()
 
	for i in range(len(dataset)):
		vector = dataset[i]
		class_value = vector[-1]
		if (class_value not in separated):
			separated[class_value] = list()
		separated[class_value].append(vector)
	return separated

# Calculate the mean of a list of numbers
def mean(numbers):
	return np.mean(numbers)

# Calculate the standard deviation of a list of numbers
def stdev(numbers):
	return np.std(numbers)

# Calculate the mean, stdev and count for each column in a dataset
def summarize_dataset(dataset):
	summaries = [(mean(column), stdev(column), len(column)) for column in zip(*dataset)]
	del(summaries[-1])
	return summaries

# Split dataset by class then calculate statistics for each row
def summarize_by_class(dataset):
	separated = separate_by_class(dataset)
	summaries = dict()
	for class_value, rows in separated.items():
		summaries[class_value] = summarize_dataset(rows)
	return summaries

# Calculate the Gaussian probability distribution function for x
def calculate_probability(x, mean, stdev):
	exponent = exp(-((x-mean)**2 / (2 * stdev**2 )))
	return (1 / (sqrt(2 * pi) * stdev)) * exponent

# Calculate the probabilities of predicting each class for a given row
def calculate_class_probabilities(summaries, row):
	total_rows = sum([summaries[label][0][2] for label in summaries])
	probabilities = dict()
	for class_value, class_summaries in summaries.items():
		probabilities[class_value] = summaries[class_value][0][2]/float(total_rows)
		for i in range(len(class_summaries)):
			mean, stdev, _ = class_summaries[i]
			probabilities[class_value] *= calculate_probability(row[i], mean, stdev)
	return probabilities

# Predict the class for a given row
def predict(summaries, row):
	probabilities = calculate_class_probabilities(summaries, row)
	best_label, best_prob = None, -1
	for class_value, probability in probabilities.items():
		if best_label is None or probability > best_prob:
			best_prob = probability
			best_label = class_value
	return best_label

 

In [170]:
# FITTING THE MODEL

model = summarize_by_class(train_dataset)

In [171]:
# TRAIN ACCURACY

y_pred_train = []

for row in X_train:
  
  # here PREDICTION hapepens for each row
  predicted = predict(model, row)
  y_pred_train.append(predicted)

score = np.sum(y_pred_train == y_train)/len(y_pred_train)

print(f'Train accuracy is {score*100} for {len(y_pred_train)} samples')

Train accuracy is 76.72253258845437 for 537 samples


In [172]:
# TEST ACCURACY
y_pred = []

for row in X_test:
  predicted_test = predict(model, row) # PREDICTION
  y_pred.append(predicted_test)

y_pred_array = np.array(y_pred)


score = np.sum(y_pred_array == y_test)/len(y_pred_array)

print(f'Train accuracy is {score*100} for {len(y_test)} samples')


Train accuracy is 76.62337662337663 for 231 samples


## Question 2 
Write a program to solve the 4-Queens problem. 
In the 4 Queens problem the object is to place 4 queens on a chessboard in such a way that no queens can capture a piece. This means that no two queens may be placed on the same row, column, or diagonal.


In [163]:

# [ '_' , '_' , '_', '_'] 
# '_' == means space not filled.. 

# here the index is the row (which row right now working on) 
# and content is the column value.. of that particluar row
# hence [1,0, 2, 3] means:
#   _ Q _ _
#   Q _ _ _
#   _ _ Q _
#   _ _ _ Q
# '_' :== this is used for representation purpose..
 
# here board is a list of 4 intgers
# row is the current row which we are working on (int)
# column is the column at which queen is placed.. (int)

# function to check if its safe to place the queen .. 
# so queen can move vertical , horizontal and diagonal . hence we checking for if all 3 are ok


def safePositionCheck(Board, row , column):
  if row==0:
    return True

  # checking if if we have queen in the row which we are looking at
  # which returns false and means 'its not safe' 
  for i in range(row):
    if Board[i]==column:
      return False
  
  # finding any two queens are on the same diagonal..
  i=row
  j=column
  while i>=0 and j>=0:
    if Board[i]==j:
      return False
    i=i-1
    j=j-1
  
  # finding any two queens are on the other same diagonal.. 
  i=row
  j=column
  while i>=0 and j<4:
    if Board[i]==j:
      return False
    i=i-1
    j=j+1

  # if all the satements are false means its safer to put a queen in the particular place..
  # so returns true..
  return True

# [1, 3 ,0 , 2]

def showResult(board):
  # index is row number
  # content is column number

   # just to display the 4 queens in the final placement..
  print('\n solution : {}\n'.format(board))
  for i in range(len(board)):
    
    if(board[i]==0):
      print('Q _ _ _ \n')
    if(board[i]==1):
      print('_ Q _ _ \n')
    if(board[i]==2):
      print('_ _ Q  _ \n')
    if(board[i]==3):
      print('_ _ _ Q \n')
  

# here stack is used along with recursion..

# here it goes down all deep to find an answer
def DFS(Board , n):
  # print('check 1')
  if n>=4:
    # only when 4 columns are filled with correct values.. 
    # that is if 4 queens can be filled.. only print the result..
    showResult(Board)
    return
  # print('check 2')
  for i in range(4):
    # print('check 3')
    if safePositionCheck(Board, n, i):
      Board[n] = i
      print(Board)

      DFS(Board, n+1) # recursion

      Board[n] = '_'


In [164]:
# Depth First Search call to find the 4 Queens positions

Board =['_', '_', '_', '_']

print('4 Queens using Depth First Search (DFS) \n ')
DFS(Board,0)

4 Queens using Depth First Search (DFS) 
 
[0, '_', '_', '_']
[0, 2, '_', '_']
[0, 3, '_', '_']
[0, 3, 1, '_']
[1, '_', '_', '_']
[1, 3, '_', '_']
[1, 3, 0, '_']
[1, 3, 0, 2]

 solution : [1, 3, 0, 2]

_ Q _ _ 

_ _ _ Q 

Q _ _ _ 

_ _ Q  _ 

[2, '_', '_', '_']
[2, 0, '_', '_']
[2, 0, 3, '_']
[2, 0, 3, 1]

 solution : [2, 0, 3, 1]

_ _ Q  _ 

Q _ _ _ 

_ _ _ Q 

_ Q _ _ 

[3, '_', '_', '_']
[3, 0, '_', '_']
[3, 0, 2, '_']
[3, 1, '_', '_']
